# LGBM - Training

Sources
1. [LGBM & Deberta Explained by ZULQARNAIN ALI](https://www.kaggle.com/code/zulqarnainalipk/lgbm-deberta-explained)

## Setup

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
import lightgbm as lgb
import pickle as pkl
import wandb
from lightgbm import log_evaluation, early_stopping
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold

In [2]:
os.chdir("../../")

In [3]:
from lib.config import config
from lib.paths import Paths
from lib.utils.utils import seed_everything
from lib.criterion.metrics import log_metrics
from lib.model.utils import quadratic_weighted_kappa, qwk_obj, get_score

In [4]:
seed_everything()

### Setting Up WandB

In [5]:
WANDB_PROJECT = "Kaggle_ASE_2.0"
WANDB_NAME = f"LGBM-ASE-9"

In [6]:
wandb.login(key=os.environ.get('WANDB_API_KEY'))
wandb.init(
    project=WANDB_PROJECT,
    name=WANDB_NAME,
    save_code=True,
    job_type="train",
    config=config,
)
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shakleenishfar (laplacesdemon43). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ishfar/.netrc


## Model Training

### Dataset Creation

In [7]:
df = pd.read_csv(Paths.FEATURE_ENGG_CSV_PATH)
df.shape

(17307, 299)

In [8]:
ids = df.loc[:, "essay_id"]
X, y = df.drop(columns=["score", "essay_id"]), df.loc[:, "score"]
del df

X.shape, y.shape

((17307, 297), (17307,))

### Callbacks

In [9]:
callbacks = [
    log_evaluation(period=25),
    early_stopping(stopping_rounds=75, first_metric_only=True),
]

### Constants

In [10]:
a = config.lgbm_a
b = config.lgbm_b

### Training Loop

In [11]:
def train_loop(X_train, y_train, X_valid, y_valid):
    y_train = y_train.astype(np.float32) - a
    y_valid = y_valid.astype(np.float32) - a

    model = lgb.LGBMRegressor(
        objective=qwk_obj,
        metrics="None",
        learning_rate=0.01,
        max_depth=5,
        num_leaves=10,
        colsample_bytree=0.3,
        reg_alpha=0.7,
        reg_lambda=0.1,
        n_estimators=700,
        random_state=config.random_seed,
        extra_trees=True,
        class_weight="balanced",
        n_jobs=6,
        verbosity=0,
    )

    # Train model
    predictor = model.fit(
        X_train,
        y_train,
        eval_names=["train", "valid"],
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks,
    )

    return model, predictor

In [12]:
def validate_model(idx, X_valid, y_valid, id_valid, predictor):
    y_valid_int = y_valid.astype(np.int64).to_numpy()
    preds = predictor.predict(X_valid) + a
    preds = preds.clip(1, 6).round().astype(np.int64)

    oof = pd.DataFrame(
        {
            "essay_id": id_valid,
            "score": y_valid_int - 1,
            "pred_score": preds - 1,
        }
    )
    log_metrics(oof, f"Fold {idx}")

    return oof

### Fold Loop

In [13]:
skf = StratifiedKFold(
    n_splits=config.lgbm_n_folds,
    random_state=config.random_seed,
    shuffle=True,
)

In [14]:
oof_df = pd.DataFrame()

for idx, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_valid, y_valid = X.loc[valid_idx], y.loc[valid_idx]
    id_valid = ids.loc[valid_idx].to_numpy()

    model, predictor = train_loop(X_train, y_train, X_valid, y_valid)

    with open(f"output/LGBM/{idx}.pkl", "wb") as file:
        pkl.dump({"predictor": predictor, "model": model}, file)

    oof = validate_model(idx, X_valid, y_valid, id_valid, predictor)
    oof_df = pd.concat([oof_df, oof])

[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.433068	valid's QWK: 0.453181
[50]	train's QWK: 0.737055	valid's QWK: 0.754012
[75]	train's QWK: 0.798599	valid's QWK: 0.811799
[100]	train's QWK: 0.838116	valid's QWK: 0.846833
[125]	train's QWK: 0.859684	valid's QWK: 0.86327
[150]	train's QWK: 0.869297	valid's QWK: 0.878138
[175]	train's QWK: 0.873054	valid's QWK: 0.882727
[200]	train's QWK: 0.87601	valid's QWK: 0.889005
[225]	train's QWK: 0.879133	valid's QWK: 0.890358
[250]	train's QWK: 0.880839	valid's QWK: 0.891393
[275]	train's QWK: 0.882086	valid's QWK: 0.892983
[300]	train's QWK: 0.882908	valid's QWK: 0.893411
[325]	train's QWK: 0.883796	valid's QWK: 0.893411
[350]	train's QWK: 0.884465	valid's QWK: 0.894965
[375]	train's QWK: 0.884932	valid's QWK: 0.893909
[400]	train's QWK: 0.885352	valid's QWK: 0.894784
[425]	train's QWK: 0.88595	valid's QWK: 0.89564
[450]	train's QWK: 0.886498	valid's QWK:

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.426811	valid's QWK: 0.42107
[50]	train's QWK: 0.736773	valid's QWK: 0.731004
[75]	train's QWK: 0.802168	valid's QWK: 0.793421
[100]	train's QWK: 0.839992	valid's QWK: 0.836942
[125]	train's QWK: 0.859912	valid's QWK: 0.863887
[150]	train's QWK: 0.86871	valid's QWK: 0.864731
[175]	train's QWK: 0.872301	valid's QWK: 0.87086
[200]	train's QWK: 0.874933	valid's QWK: 0.872307
[225]	train's QWK: 0.877049	valid's QWK: 0.875241
[250]	train's QWK: 0.880077	valid's QWK: 0.881095
[275]	train's QWK: 0.882125	valid's QWK: 0.883398
[300]	train's QWK: 0.88328	valid's QWK: 0.882498
[325]	train's QWK: 0.884608	valid's QWK: 0.883565
[350]	train's QWK: 0.885443	valid's QWK: 0.885598
[375]	train's QWK: 0.885985	valid's QWK: 0.88592
[400]	train's QWK: 0.886506	valid's QWK: 0.886654
[425]	train's QWK: 0.88663	valid's QWK: 0.888434
[450]	train's QWK: 0.886837	valid's QWK: 0

In [15]:
score = get_score(oof_df["score"], oof_df["pred_score"])
print(f'Overall Score: {score:<.4f}')
wandb.log({"CV/qwk_score": score})

Overall Score: 0.8858


In [16]:
wandb.log({"oof_table": wandb.Table(dataframe=oof_df)})

In [17]:
log_metrics(
    pd.DataFrame(
        {
            "score": oof_df.score.to_numpy(),
            "pred_score": oof_df.pred_score.to_numpy(),
        }
    ),
    "Overall",
)

In [18]:
wandb.finish()

CV/qwk_score,▁
CV/qwk_score,0.88578
